In [ ]:
!pip install xlsxwriter

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive/')

Drive already mounted at /content/MyDrive/; to attempt to forcibly remount, call drive.mount("/content/MyDrive/", force_remount=True).


In [ ]:
from glob import glob
import pandas as pd
import numpy as np
import os
import re

pd.set_option('display.max_columns', 50)

In [ ]:
import os
os.chdir('/content/MyDrive/MyDrive/Colab Notebooks')

In [ ]:
os.getcwd()

'/content/MyDrive/MyDrive/Colab Notebooks'

In [ ]:
def get_conditions(fpath):
    conditions=fpath.split('/')[-2]
    dataset=conditions.split('-')[0]
    loc_unit=conditions.split('-')[1]
    netps=re.findall('(?<=netps)[-]?[0-9]+', conditions)[0]
    nls=re.findall('(?<=nls)[-]?[0-9]+', conditions)[0]
    nps=re.findall('(?<=nps)[-]?[0-9]+', conditions)[0]
    model=re.findall('(?<=model)[^-]+', conditions)[0]
    return dataset, loc_unit, netps, nls, nps, model

In [ ]:
resCols = pd.MultiIndex.from_tuples([(  'hyperparameters',                       'discriminator model'),
            (  'hyperparameters',                                    'method'),
            (  'hyperparameters', 'selection_criterion (best step selection)'),
            (  'hyperparameters',                                   'epsilon'),
            (  'hyperparameters',                               'optim_steps'),
            (  'hyperparameters',                          'early_stop_steps'),
            (  'hyperparameters',                          'locate_how_often'),
            (  'hyperparameters',                      'locate_toks_per_step'),
            (  'hyperparameters',                               'locate_unit'),
            (          'fluency',                               'gpt2-xl ppl'),
            (          'fluency',                         'gpt2-xl total ppl'),
            (  'perspective api',                          'avg_max_toxicity'),
            (  'perspective api',                    ' toxicity prob prompt '),
            (  'perspective api',                            ' avg_toxicity '),
            (  'perspective api',                            ' toxicity prob'),
            (     'energy model',                          'avg_max_toxicity'),
            (     'energy model',                    ' toxicity prob prompt '),
            (     'energy model',                            ' avg_toxicity '),
            (     'energy model',                            ' toxicity prob'),
            ('mucola classifier',                          'avg_max_toxicity'),
            ('mucola classifier',                    ' toxicity prob prompt '),
            ('mucola classifier',                            ' avg_toxicity '),
            ('mucola classifier',                            ' toxicity prob'),
            (        'diversity',                                   'dist-1 '),
            (        'diversity',                                   'dist-2 '),
            (        'diversity',                                   'dist-3 ')])

# Metrics

## Result concatenation

In [ ]:
results_file_list=glob('outputs/toxicity/locate-unit/*/results_epsilon-3.txt')

In [ ]:
jigsaw_res_list, gpt2_res_list=[], []
for sample in results_file_list:
    dataset, loc_unit, netps, nls, nps, model=get_conditions(sample)
    try:
        # print(dataset, loc_unit, netps, nls, nps, model)

        with open(sample, 'r') as f:
            res = f.readlines()
        ppl, total_ppl = res[0].strip().split('= ')[1].split(', ')
        api_tox_res = [x.split(',')[0].strip() for x in res[1].strip().split('=')[1:]]
        energy_tox_res = [x.split(',')[0].strip() for x in res[2].strip().split('=')[1:]]
        mucola_tox_res = [x.split(',')[0].strip() for x in res[3].strip().split('=')[1:]]
        dist1=res[4].split('=')[1].strip()
        dist2=res[5].split('=')[1].strip()
        dist3=res[6].split('=')[1].strip()

        tmp_res = [model, 'locate&edit', 'allsat', -3.0, 20, 4, nls, netps, loc_unit]+[ppl, total_ppl, *api_tox_res, *energy_tox_res, *mucola_tox_res, dist1, dist2, dist3]

        if dataset=="jigsaw":
            jigsaw_res_list.append(tmp_res)
        else:
            gpt2_res_list.append(tmp_res)
    except:
        print(sample)
        tmp_res = [model, 'locate&edit', 'allsat', -3.0, 20, 4, nls, netps, loc_unit]

        if dataset=="jigsaw":
            jigsaw_res_list.append(tmp_res)
        else:
            gpt2_res_list.append(tmp_res)

In [ ]:
jigsaw_res_tb=pd.DataFrame(jigsaw_res_list, columns=resCols)

In [ ]:
jigsaw_res_tb

hyperparameters                                                         \
   discriminator model       method selection_criterion (best step selection)   
0               energy  locate&edit                                    allsat   
1               energy  locate&edit                                    allsat   
2               energy  locate&edit                                    allsat   
3               energy  locate&edit                                    allsat   
4               energy  locate&edit                                    allsat   
5               energy  locate&edit                                    allsat   
6               energy  locate&edit                                    allsat   
7               energy  locate&edit                                    allsat   
8               energy  locate&edit                                    allsat   
9               mucola  locate&edit                                    allsat   
10              mucola  locate&edit                                    allsat   
11              mucola  locate&edit                                    allsat   
12              mucola  locate&edit                                    allsat   
13              mucola  locate&edit                                    allsat   
14              mucola  locate&edit                                    allsat   
15              mucola  locate&edit                                    allsat   
16              mucola  locate&edit                                    allsat   
17              mucola  locate&edit                                    allsat   

                                                                               \
   epsilon optim_steps early_stop_steps locate_how_often locate_toks_per_step   
0     -3.0          20                4                1                    1   
1     -3.0          20                4                1                    1   
2     -3.0          20                4                1                    3   
3     -3.0          20                4                1                    3   
4     -3.0          20                4                4                    3   
5     -3.0          20                4                4                    1   
6     -3.0          20                4                4                    1   
7     -3.0          20                4                4                    3   
8     -3.0          20                4               -1                   -1   
9     -3.0          20                4                1                    1   
10    -3.0          20                4                1                    3   
11    -3.0          20                4                4                    1   
12    -3.0          20                4                1                    1   
13    -3.0          20                4                1                    3   
14    -3.0          20                4                4                    1   
15    -3.0          20                4                4                    3   
16    -3.0          20                4                4                    3   
17    -3.0          20                4               -1                   -1   

                           fluency                           perspective api  \
   locate_unit         gpt2-xl ppl   gpt2-xl total ppl      avg_max_toxicity   
0        token   566.4588731595949                 nan    0.1255748766107143   
1         word  323.03734110425313                 nan   0.12571012613505103   
2        token   828.7293603402319                 nan   0.05983951393765306   
3         word   954.0987700927151                 nan  0.058202691890784176   
4        token   1302.333698846152                 nan   0.08820311425984693   
5        token  440.32303605574543                 nan   0.15684425978831631   
6         word  459.95071636397324                 nan    0.1452701707341837   
7         word  437.00664206149145           

In [ ]:
gpt2_res_tb=pd.DataFrame(gpt2_res_list, columns=resCols)

In [ ]:
gpt2_res_tb

hyperparameters                                                         \
   discriminator model       method selection_criterion (best step selection)   
0               energy  locate&edit                                    allsat   
1               energy  locate&edit                                    allsat   
2               energy  locate&edit                                    allsat   
3               energy  locate&edit                                    allsat   
4               energy  locate&edit                                    allsat   
5               energy  locate&edit                                    allsat   
6               energy  locate&edit                                    allsat   
7               energy  locate&edit                                    allsat   
8               mucola  locate&edit                                    allsat   
9               mucola  locate&edit                                    allsat   
10              mucola  locate&edit                                    allsat   
11              mucola  locate&edit                                    allsat   
12              mucola  locate&edit                                    allsat   
13              mucola  locate&edit                                    allsat   
14              mucola  locate&edit                                    allsat   
15              mucola  locate&edit                                    allsat   
16              energy  locate&edit                                    allsat   
17              mucola  locate&edit                                    allsat   

                                                                               \
   epsilon optim_steps early_stop_steps locate_how_often locate_toks_per_step   
0     -3.0          20                4                1                    1   
1     -3.0          20                4                1                    1   
2     -3.0          20                4                1                    3   
3     -3.0          20                4                4                    1   
4     -3.0          20                4                4                    1   
5     -3.0          20                4                4                    3   
6     -3.0          20                4                4                    3   
7     -3.0          20                4                1                    3   
8     -3.0          20                4                1                    1   
9     -3.0          20                4                1                    3   
10    -3.0          20                4                4                    1   
11    -3.0          20                4                4                    3   
12    -3.0          20                4                1                    1   
13    -3.0          20                4                1                    3   
14    -3.0          20                4                4                    1   
15    -3.0          20                4                4                    3   
16    -3.0          20                4               -1                   -1   
17    -3.0          20                4               -1                   -1   

                           fluency                          perspective api  \
   locate_unit         gpt2-xl ppl   gpt2-xl total ppl     avg_max_toxicity   
0        token   862.8069928863146   54.93418246015284       0.276296888132   
1         word   356.4129388629249    55.0221867743377  0.26596619920000003   
2        token   955.3760259361036  100.88047884283306  0.24152285095200002   
3        token   841.4751980812048   49.03408910597814       0.290709280476   
4         word   147.8478637378549   50.20328951156568       0.277183825888   
5        token   952.8417710562613   90.36653991132312  0.24939685452000002   
6         word  233.41646978132022   89.95280241629155       0.233118249516   
7         word  231.92518036858596   98.05205520027705

## Total perplexity treatment (jigsaw)

In [ ]:
dirlists=glob('outputs/toxicity/locate-unit/jigsaw-*/results_epsilon-3.txt.ppl-big')

In [ ]:
totppls=[]
for _dir in dirlists:
    res=pd.read_csv(_dir,names=['ppl', 'total_nll', 'total_tokens'])
    res=res.astype(float)
    dataset, loc_unit, netps, nls, nps, model=get_conditions(_dir)
    tmp_res = [model, 'locate&edit', 'allsat', -3.0, 20, 4, nls, netps, loc_unit]+[np.exp(res.total_nll.sum()/res.total_tokens.sum())]
    totppls.append(tmp_res)

In [ ]:
totPplCols = pd.MultiIndex.from_tuples([(  'hyperparameters',                       'discriminator model'),
            (  'hyperparameters',                                    'method'),
            (  'hyperparameters', 'selection_criterion (best step selection)'),
            (  'hyperparameters',                                   'epsilon'),
            (  'hyperparameters',                               'optim_steps'),
            (  'hyperparameters',                          'early_stop_steps'),
            (  'hyperparameters',                          'locate_how_often'),
            (  'hyperparameters',                      'locate_toks_per_step'),
            (  'hyperparameters',                               'locate_unit'),
            (          'fluency',                         'gpt2-xl total ppl')])

In [ ]:
totppls=pd.DataFrame(totppls,columns=totPplCols)

In [ ]:
totppls.sort_values(by=[('hyperparameters', 'discriminator model'), ('hyperparameters', 'locate_how_often'), ('hyperparameters', 'locate_toks_per_step')])

hyperparameters                                                         \
   discriminator model       method selection_criterion (best step selection)   
8               energy  locate&edit                                    allsat   
0               energy  locate&edit                                    allsat   
1               energy  locate&edit                                    allsat   
2               energy  locate&edit                                    allsat   
3               energy  locate&edit                                    allsat   
5               energy  locate&edit                                    allsat   
6               energy  locate&edit                                    allsat   
4               energy  locate&edit                                    allsat   
7               energy  locate&edit                                    allsat   
17              mucola  locate&edit                                    allsat   
9               mucola  locate&edit                                    allsat   
12              mucola  locate&edit                                    allsat   
10              mucola  locate&edit                                    allsat   
13              mucola  locate&edit                                    allsat   
11              mucola  locate&edit                                    allsat   
14              mucola  locate&edit                                    allsat   
15              mucola  locate&edit                                    allsat   
16              mucola  locate&edit                                    allsat   

                                                                               \
   epsilon optim_steps early_stop_steps locate_how_often locate_toks_per_step   
8     -3.0          20                4               -1                   -1   
0     -3.0          20                4                1                    1   
1     -3.0          20                4                1                    1   
2     -3.0          20                4                1                    3   
3     -3.0          20                4                1                    3   
5     -3.0          20                4                4                    1   
6     -3.0          20                4                4                    1   
4     -3.0          20                4                4                    3   
7     -3.0          20                4                4                    3   
17    -3.0          20                4               -1                   -1   
9     -3.0          20                4                1                    1   
12    -3.0          20                4                1                    1   
10    -3.0          20                4                1                    3   
13    -3.0          20                4                1                    3   
11    -3.0          20                4                4                    1   
14    -3.0          20                4                4                    1   
15    -3.0          20                4                4                    3   
16    -3.0          20                4                4                    3   

                         fluency  
   locate_unit gpt2-xl total ppl  
8         word          4.035142  
0        token         75.874977  
1         word         73.869331  
2        token        123.871131  
3         word        121.129310  
5        token         62.647892  
6         word         64.042137  
4        token         93.782506  
7         word         93.473724  
17        word          3.380330  
9         word        113.251305  
12       token        115.111815  
10        word        177.714705  
13       token        197.483871  
11        word         76.503122  
14       token         75.220103  
15        word        124.283858  
16       token        127.909155

In [ ]:
jigsaw_res_tb.drop(columns=(          'fluency',                         'gpt2-xl total ppl'),inplace=True)

In [ ]:
jigsaw_res_tb=jigsaw_res_tb.merge(totppls,how='left',on=[(  'hyperparameters',                       'discriminator model'),
            (  'hyperparameters',                                    'method'),
            (  'hyperparameters', 'selection_criterion (best step selection)'),
            (  'hyperparameters',                                   'epsilon'),
            (  'hyperparameters',                               'optim_steps'),
            (  'hyperparameters',                          'early_stop_steps'),
            (  'hyperparameters',                          'locate_how_often'),
            (  'hyperparameters',                      'locate_toks_per_step'),
            (  'hyperparameters',                               'locate_unit')])

In [ ]:
jigsaw_res_tb

hyperparameters                                                         \
   discriminator model       method selection_criterion (best step selection)   
0               energy  locate&edit                                    allsat   
1               energy  locate&edit                                    allsat   
2               energy  locate&edit                                    allsat   
3               energy  locate&edit                                    allsat   
4               energy  locate&edit                                    allsat   
5               energy  locate&edit                                    allsat   
6               energy  locate&edit                                    allsat   
7               energy  locate&edit                                    allsat   
8               energy  locate&edit                                    allsat   
9               mucola  locate&edit                                    allsat   
10              mucola  locate&edit                                    allsat   
11              mucola  locate&edit                                    allsat   
12              mucola  locate&edit                                    allsat   
13              mucola  locate&edit                                    allsat   
14              mucola  locate&edit                                    allsat   
15              mucola  locate&edit                                    allsat   
16              mucola  locate&edit                                    allsat   
17              mucola  locate&edit                                    allsat   

                                                                               \
   epsilon optim_steps early_stop_steps locate_how_often locate_toks_per_step   
0     -3.0          20                4                1                    1   
1     -3.0          20                4                1                    1   
2     -3.0          20                4                1                    3   
3     -3.0          20                4                1                    3   
4     -3.0          20                4                4                    3   
5     -3.0          20                4                4                    1   
6     -3.0          20                4                4                    1   
7     -3.0          20                4                4                    3   
8     -3.0          20                4               -1                   -1   
9     -3.0          20                4                1                    1   
10    -3.0          20                4                1                    3   
11    -3.0          20                4                4                    1   
12    -3.0          20                4                1                    1   
13    -3.0          20                4                1                    3   
14    -3.0          20                4                4                    1   
15    -3.0          20                4                4                    3   
16    -3.0          20                4                4                    3   
17    -3.0          20                4               -1                   -1   

                           fluency       perspective api  \
   locate_unit         gpt2-xl ppl      avg_max_toxicity   
0        token   566.4588731595949    0.1255748766107143   
1         word  323.03734110425313   0.12571012613505103   
2        token   828.7293603402319   0.05983951393765306   
3         word   954.0987700927151  0.058202691890784176   
4        token   1302.333698846152   0.08820311425984693   
5        token  440.32303605574543   0.15684425978831631   
6         word  459.95071636397324    0.1452701707341837   
7         word  437.00664206149145    0.0832902905882653   
8         word   8.656188354459573         0.03144569798   
9         word    796.587823223884   0.11836320204785714   
10        word   603.9118684779904   0.062371

In [ ]:
jigsaw_res_tb=jigsaw_res_tb[resCols]

In [ ]:
jigsaw_res_tb

hyperparameters                                                         \
   discriminator model       method selection_criterion (best step selection)   
0               energy  locate&edit                                    allsat   
1               energy  locate&edit                                    allsat   
2               energy  locate&edit                                    allsat   
3               energy  locate&edit                                    allsat   
4               energy  locate&edit                                    allsat   
5               energy  locate&edit                                    allsat   
6               energy  locate&edit                                    allsat   
7               energy  locate&edit                                    allsat   
8               energy  locate&edit                                    allsat   
9               mucola  locate&edit                                    allsat   
10              mucola  locate&edit                                    allsat   
11              mucola  locate&edit                                    allsat   
12              mucola  locate&edit                                    allsat   
13              mucola  locate&edit                                    allsat   
14              mucola  locate&edit                                    allsat   
15              mucola  locate&edit                                    allsat   
16              mucola  locate&edit                                    allsat   
17              mucola  locate&edit                                    allsat   

                                                                               \
   epsilon optim_steps early_stop_steps locate_how_often locate_toks_per_step   
0     -3.0          20                4                1                    1   
1     -3.0          20                4                1                    1   
2     -3.0          20                4                1                    3   
3     -3.0          20                4                1                    3   
4     -3.0          20                4                4                    3   
5     -3.0          20                4                4                    1   
6     -3.0          20                4                4                    1   
7     -3.0          20                4                4                    3   
8     -3.0          20                4               -1                   -1   
9     -3.0          20                4                1                    1   
10    -3.0          20                4                1                    3   
11    -3.0          20                4                4                    1   
12    -3.0          20                4                1                    1   
13    -3.0          20                4                1                    3   
14    -3.0          20                4                4                    1   
15    -3.0          20                4                4                    3   
16    -3.0          20                4                4                    3   
17    -3.0          20                4               -1                   -1   

                           fluency                         perspective api  \
   locate_unit         gpt2-xl ppl gpt2-xl total ppl      avg_max_toxicity   
0        token   566.4588731595949         75.874977    0.1255748766107143   
1         word  323.03734110425313         73.869331   0.12571012613505103   
2        token   828.7293603402319        123.871131   0.05983951393765306   
3         word   954.0987700927151        121.129310  0.058202691890784176   
4        token   1302.333698846152         93.782506   0.08820311425984693   
5        token  440.32303605574543         62.647892   0.15684425978831631   
6         word  459.95071636397324         64.042137    0.1452701707341837   
7         word  437.00664206149145         93.473724    0.08329

## Add results of original (unmutated) results

In [ ]:
results_file_list=glob('ell-e/toxicity-avoidance/data/results_*.txt')

In [ ]:
jigsaw_res_list, gpt2_res_list=[], []
for sample in results_file_list:

    dataset=sample.split('/')[-1].split('_')[-1].split('.txt')[0]

    with open(sample, 'r') as f:
        res = f.readlines()
    ppl, total_ppl = res[0].strip().split('= ')[1].split(', ')
    api_tox_res = [x.split(',')[0].strip() for x in res[1].strip().split('=')[1:]]
    energy_tox_res = [x.split(',')[0].strip() for x in res[2].strip().split('=')[1:]]
    mucola_tox_res = [x.split(',')[0].strip() for x in res[3].strip().split('=')[1:]]
    dist1=res[4].split('=')[1].strip()
    dist2=res[5].split('=')[1].strip()
    dist3=res[6].split('=')[1].strip()

    tmp_res = ['n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a']+[ppl, total_ppl, *api_tox_res, *energy_tox_res, *mucola_tox_res, dist1, dist2, dist3]

    if dataset=="jigsaw":
        jigsaw_res_list.append(tmp_res)
    else:
        gpt2_res_list.append(tmp_res)

In [ ]:
jigsaw_res_tb_original = pd.DataFrame(jigsaw_res_list,columns=resCols)
gpt2_res_tb_original = pd.DataFrame(gpt2_res_list,columns=resCols)

In [ ]:
jigsaw_res_tb = pd.concat([jigsaw_res_tb,jigsaw_res_tb_original],axis=0,ignore_index=True)

In [ ]:
gpt2_res_tb = pd.concat([gpt2_res_tb,gpt2_res_tb_original],axis=0,ignore_index=True)

In [ ]:
jigsaw_res_tb

hyperparameters                                                         \
   discriminator model       method selection_criterion (best step selection)   
0               energy  locate&edit                                    allsat   
1               energy  locate&edit                                    allsat   
2               energy  locate&edit                                    allsat   
3               energy  locate&edit                                    allsat   
4               energy  locate&edit                                    allsat   
5               energy  locate&edit                                    allsat   
6               energy  locate&edit                                    allsat   
7               energy  locate&edit                                    allsat   
8               energy  locate&edit                                    allsat   
9               mucola  locate&edit                                    allsat   
10              mucola  locate&edit                                    allsat   
11              mucola  locate&edit                                    allsat   
12              mucola  locate&edit                                    allsat   
13              mucola  locate&edit                                    allsat   
14              mucola  locate&edit                                    allsat   
15              mucola  locate&edit                                    allsat   
16              mucola  locate&edit                                    allsat   
17              mucola  locate&edit                                    allsat   
18                 n/a          n/a                                       n/a   

                                                                               \
   epsilon optim_steps early_stop_steps locate_how_often locate_toks_per_step   
0     -3.0          20                4                1                    1   
1     -3.0          20                4                1                    1   
2     -3.0          20                4                1                    3   
3     -3.0          20                4                1                    3   
4     -3.0          20                4                4                    3   
5     -3.0          20                4                4                    1   
6     -3.0          20                4                4                    1   
7     -3.0          20                4                4                    3   
8     -3.0          20                4               -1                   -1   
9     -3.0          20                4                1                    1   
10    -3.0          20                4                1                    3   
11    -3.0          20                4                4                    1   
12    -3.0          20                4                1                    1   
13    -3.0          20                4                1                    3   
14    -3.0          20                4                4                    1   
15    -3.0          20                4                4                    3   
16    -3.0          20                4                4                    3   
17    -3.0          20                4               -1                   -1   
18     n/a         n/a              n/a              n/a                  n/a   

                           fluency                          perspective api  \
   locate_unit         gpt2-xl ppl  gpt2-xl total ppl      avg_max_toxicity   
0        token   566.4588731595949          75.874977    0.1255748766107143   
1         word  323.03734110425313          73.869331   0.12571012613505103   
2        token   828.7293603402319         123.871131   0.05983951393765306   
3         word   954.0987700927151          121.12931  0.058202691890784176   
4        token   1302.333698846152          93.782506   0.08820311425984693   
5        token  440.32303605574543          62.647

In [ ]:
gpt2_res_tb

hyperparameters                                                         \
   discriminator model       method selection_criterion (best step selection)   
0               energy  locate&edit                                    allsat   
1               energy  locate&edit                                    allsat   
2               energy  locate&edit                                    allsat   
3               energy  locate&edit                                    allsat   
4               energy  locate&edit                                    allsat   
5               energy  locate&edit                                    allsat   
6               energy  locate&edit                                    allsat   
7               energy  locate&edit                                    allsat   
8               mucola  locate&edit                                    allsat   
9               mucola  locate&edit                                    allsat   
10              mucola  locate&edit                                    allsat   
11              mucola  locate&edit                                    allsat   
12              mucola  locate&edit                                    allsat   
13              mucola  locate&edit                                    allsat   
14              mucola  locate&edit                                    allsat   
15              mucola  locate&edit                                    allsat   
16              energy  locate&edit                                    allsat   
17              mucola  locate&edit                                    allsat   
18                 n/a          n/a                                       n/a   

                                                                               \
   epsilon optim_steps early_stop_steps locate_how_often locate_toks_per_step   
0     -3.0          20                4                1                    1   
1     -3.0          20                4                1                    1   
2     -3.0          20                4                1                    3   
3     -3.0          20                4                4                    1   
4     -3.0          20                4                4                    1   
5     -3.0          20                4                4                    3   
6     -3.0          20                4                4                    3   
7     -3.0          20                4                1                    3   
8     -3.0          20                4                1                    1   
9     -3.0          20                4                1                    3   
10    -3.0          20                4                4                    1   
11    -3.0          20                4                4                    3   
12    -3.0          20                4                1                    1   
13    -3.0          20                4                1                    3   
14    -3.0          20                4                4                    1   
15    -3.0          20                4                4                    3   
16    -3.0          20                4               -1                   -1   
17    -3.0          20                4               -1                   -1   
18     n/a         n/a              n/a              n/a                  n/a   

                           fluency                          perspective api  \
   locate_unit         gpt2-xl ppl   gpt2-xl total ppl     avg_max_toxicity   
0        token   862.8069928863146   54.93418246015284       0.276296888132   
1         word   356.4129388629249    55.0221867743377  0.26596619920000003   
2        token   955.3760259361036  100.88047884283306  0.24152285095200002   
3        token   841.4751980812048   49.03408910597814       0.290709280476   
4         word   147.8478637378549   50.20328951156568       0.277183825888   
5        token   952.8417710562613   90.3665399113

## Save the result

In [ ]:
os.makedirs('evaluate',exist_ok=True)

In [ ]:
writer = pd.ExcelWriter('evaluate/experiments-final.xlsx', mode='w',engine = 'xlsxwriter')
gpt2_res_tb.to_excel(writer, sheet_name = 'gpt2_results')
jigsaw_res_tb.to_excel(writer, sheet_name = 'jigsaw_results')
writer.close()

# Qualitative Results

In [1]:
from glob import glob
from google.colab import drive
drive.mount('/content/MyDrive')

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [2]:
import os
os.chdir('/content/MyDrive/MyDrive/Colab Notebooks')

In [3]:
import pandas as pd

## Save as an excel file

A key point is to align original samples with the corresponding edited versions.

### GPT2 dataset

In [45]:
flist=['/content/MyDrive/MyDrive/Colab Notebooks/ell-e/toxicity-avoidance/data/testset_gpt2_2500.jsonl']
flist+=glob('/content/MyDrive/MyDrive/Colab Notebooks/outputs/toxicity/locate-unit/gpt2*/outputs_epsilon-3.txt')

In [46]:
res_df=pd.DataFrame()

In [48]:
for myf in flist:
  myf_name = myf.split('/')[-2]
  print(myf_name)
  mydf = pd.read_json(myf, lines=True)
  mydf.prompt = mydf.prompt.apply(lambda x: x['text'])
  mydf.generations= mydf.generations.apply(lambda x: [_x['text'] for _x in x])
  mydf = pd.merge(mydf.prompt, mydf.generations.explode(), left_index=True, right_index=True)
  mydf = mydf.rename(columns={'generations': myf_name})
  if res_df.empty:
    res_df=mydf
  else:
    try:
      res_df=pd.concat([res_df,mydf],axis=1)
    except:
      print(len(set(res_df.index)))
      res_df.head()
      print(len(set(mydf.index)))
      mydf.head()

data
gpt2-token-netps1-nls1-nps1-modelenergy
gpt2-word-netps1-nls1-nps1-modelenergy
gpt2-token-netps3-nls1-nps1-modelenergy
gpt2-token-netps1-nls4-nps1-modelenergy
gpt2-word-netps1-nls4-nps1-modelenergy
gpt2-token-netps3-nls4-nps1-modelenergy
gpt2-word-netps3-nls4-nps1-modelenergy
gpt2-word-netps3-nls1-nps1-modelenergy
gpt2-word-netps1-nls1-nps1-modelmucola
gpt2-word-netps3-nls1-nps1-modelmucola
gpt2-word-netps1-nls4-nps1-modelmucola
gpt2-word-netps3-nls4-nps1-modelmucola
gpt2-token-netps1-nls1-nps1-modelmucola
gpt2-token-netps3-nls1-nps1-modelmucola
gpt2-token-netps1-nls4-nps1-modelmucola
gpt2-token-netps3-nls4-nps1-modelmucola
gpt2-word-netps-1-nls-1-nps1-modelenergy
gpt2-word-netps-1-nls-1-nps1-modelmucola


In [49]:
# check if "prompts" column in all concatenated dataframes were in the same order
criterion_index=0
for myindex in range(2, res_df.shape[1],2):
  if not (res_df.iloc[:, criterion_index] == res_df.iloc[:, myindex]).all():
    print(res_df.loc[res_df.iloc[:, criterion_index] != res_df.iloc[:, myindex]].iloc[:, [criterion_index,myindex]])

In [50]:
res_df = res_df.iloc[:, [0]+list(range(1, res_df.shape[1], 2))]

In [52]:
# # sample 50 prompts
# myf = flist[0]
# mydf = pd.read_json(myf, lines=True)
# mydf.prompt = mydf.prompt.apply(lambda x: x['text'])
# prompt_list=mydf.sample(50, random_state=0)['prompt'].tolist()

# # obtain a subset of res_df
# res_df_s=res_df.loc[res_df.prompt.isin(prompt_list),:]

# sample 3 generations for each prompt
res_df_s=res_df.groupby('prompt').sample(3, random_state=0)

In [57]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.3/154.3 kB 3.3 MB/s eta 0:00:00


In [58]:
res_df_s.to_excel('qualitative_res_gpt2_data.xlsx')

### Jigsaw dataset

In [59]:
flist=['/content/MyDrive/MyDrive/Colab Notebooks/ell-e/toxicity-avoidance/data/testset_jigsaw_1960_for_eval.jsonl']
flist+=glob('/content/MyDrive/MyDrive/Colab Notebooks/outputs/toxicity/locate-unit/jigsaw*/outputs_epsilon-3.txt')

In [60]:
res_df=pd.DataFrame()

In [61]:
for myf in flist:
  myf_name = myf.split('/')[-2]
  print(myf_name)
  mydf = pd.read_json(myf, lines=True)
  mydf.prompt = mydf.prompt.apply(lambda x: x['text'])
  mydf.generations= mydf.generations.apply(lambda x: [_x['text'] for _x in x])
  mydf = pd.merge(mydf.prompt, mydf.generations.explode(), left_index=True, right_index=True)
  mydf = mydf.rename(columns={'generations': myf_name})
  if res_df.empty:
    res_df=mydf
  else:
    try:
      res_df=pd.concat([res_df,mydf],axis=1)
    except:
      print(len(set(res_df.index)))
      res_df.head()
      print(len(set(mydf.index)))
      mydf.head()

data
jigsaw-token-netps1-nls1-nps1-modelenergy
jigsaw-word-netps1-nls1-nps1-modelenergy
jigsaw-token-netps3-nls1-nps1-modelenergy
jigsaw-word-netps3-nls1-nps1-modelenergy
jigsaw-token-netps3-nls4-nps1-modelenergy
jigsaw-token-netps1-nls4-nps1-modelenergy
jigsaw-word-netps1-nls4-nps1-modelenergy
jigsaw-word-netps3-nls4-nps1-modelenergy
jigsaw-word-netps-1-nls-1-nps1-modelenergy
jigsaw-word-netps1-nls1-nps1-modelmucola
jigsaw-word-netps3-nls1-nps1-modelmucola
jigsaw-word-netps1-nls4-nps1-modelmucola
jigsaw-token-netps1-nls1-nps1-modelmucola
jigsaw-token-netps3-nls1-nps1-modelmucola
jigsaw-token-netps1-nls4-nps1-modelmucola
jigsaw-word-netps3-nls4-nps1-modelmucola
jigsaw-token-netps3-nls4-nps1-modelmucola
jigsaw-word-netps-1-nls-1-nps1-modelmucola


In [62]:
# check if "prompts" column in all concatenated dataframes were in the same order
criterion_index=0
for myindex in range(2, res_df.shape[1],2):
  if not (res_df.iloc[:, criterion_index] == res_df.iloc[:, myindex]).all():
    print(res_df.loc[res_df.iloc[:, criterion_index] != res_df.iloc[:, myindex]].iloc[:, [criterion_index,myindex]])

     prompt         prompt
0            <|endoftext|>
1            <|endoftext|>
2            <|endoftext|>
3            <|endoftext|>
4            <|endoftext|>
...     ...            ...
1955         <|endoftext|>
1956         <|endoftext|>
1957         <|endoftext|>
1958         <|endoftext|>
1959         <|endoftext|>

[1960 rows x 2 columns]
     prompt         prompt
0            <|endoftext|>
1            <|endoftext|>
2            <|endoftext|>
3            <|endoftext|>
4            <|endoftext|>
...     ...            ...
1955         <|endoftext|>
1956         <|endoftext|>
1957         <|endoftext|>
1958         <|endoftext|>
1959         <|endoftext|>

[1960 rows x 2 columns]
     prompt         prompt
0            <|endoftext|>
1            <|endoftext|>
2            <|endoftext|>
3            <|endoftext|>
4            <|endoftext|>
...     ...            ...
1955         <|endoftext|>
1956         <|endoftext|>
1957         <|endoftext|>
1958         <|endoftext|>
1959 

In [63]:
res_df = res_df.iloc[:, [0]+list(range(1, res_df.shape[1], 2))]

In [65]:
# sample 100 generations
res_df_s=res_df.sample(100, random_state=0)

In [66]:
res_df_s.to_excel('qualitative_res_jigsaw_data.xlsx')